In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import auc, roc_curve
from catboost import CatBoostClassifier, Pool

In [2]:
def print_progressbar(total, i):
    """
    total : total iteration number.
    i : iteration count, starting from 0.
    """
    import math
    step = 25 / total

    # Print the progress bar
    print('\r' + f'Progress: '
        f"[{'=' * int((i+1) * step) + ' ' * (25 - int((i+1) * step))}]"
        f"({math.floor((i+1) * 100 / (total))} %) ({i+1}/{total})",
        end='')
    if (i+1) == total: print("")

## Example data

In [3]:
train = pd.read_csv("sample_train_df.csv", sep = ',')
test = pd.read_csv("sample_test_df.csv", sep = ',')

In [4]:
y_train = pd.read_csv("y_train_df.csv", sep = ',')
y_test = pd.read_csv("y_test_df.csv", sep = ',')

In [5]:
train.head()

,numeric1,numeric2,numeric3,numeric4,category1,category2,category3,category4,category5,category6,...,category41,category42,category43,category44,category45,category46,category47,category48,category49,category50
0,4.593655,3.791250,4.100946,50,1,0,0,1,1,1,...,1,0,0,0,0,1,0,0,0,0
1,4.642098,3.726833,3.942003,60,1,0,1,1,1,0,...,1,1,0,0,0,0,0,0,0,0
2,4.690728,4.113928,4.115909,41,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
3,4.624162,3.591310,3.840761,48,1,0,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,4.680144,3.778439,3.965647,54,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [6]:
y_train.head()

,y
0,0
1,0
2,0
3,0
4,0


## Numeric Features Split

### numeric features repositioning

In [7]:
numeric_features = ["numeric1","numeric2","numeric3","numeric4"]

In [8]:
train_numeric = train[numeric_features]
train_category = train.drop(numeric_features,1)

test_numeric = test[numeric_features]
test_category = test.drop(numeric_features,1)

In [9]:
new_train = pd.concat([train_numeric,train_category],1)
new_test = pd.concat([test_numeric,test_category],1)

## Recursive Fetures Elemination(RFE)

In [10]:
def RFE(train, y_train, test, y_test, num_round = 10, num_cut = 10):
    df = pd.DataFrame(columns=['remain_features','cut_features', 'test_AUC'])
    
    cut_features_list = []
    features = []
    test_auc_list = []
    
    ## If the numeric features are removed, the order must be changed.
    for i in range(0,num_round):
        train = train.drop(cut_features_list,1)
        test = test.drop(cut_features_list,1)
        mlist = list()
        
        for word in list(train.columns):
            for numer in numeric_features:
                if numer in word:
                    mlist.append(numer)
                    
        # catboost categorical features position
        
        categorical_features = np.array(list(range(len(mlist),train.shape[1])))
        train_pool = Pool(train,y_train, cat_features= categorical_features)

        ### This model is GPU mode
        
        cat_model = CatBoostClassifier(iterations=10,
                                bootstrap_type= 'Poisson',    
                                random_strength=10,
                                logging_level='Silent',
                                task_type="GPU",
                                eval_metric="AUC",
                                cat_features=categorical_features)
        
        
        ## Calcurate AUC
        
        cat_model.fit(train_pool)
        y_test_proba2 = cat_model.predict_proba(test)
        fpr, tpr, thresholds3 = roc_curve(y_test ,y_test_proba2[:,1])
        test_auc = auc(fpr, tpr)

        
        ## remain features name
        features.append(train.columns)

        ## feature importance caculate
        col = train.columns
        col = pd.DataFrame(cat_model.get_feature_importance()).set_index(col).sort_values(by=0)
        col = col[col>0].dropna()
        
        ## cut features name
        cut_features_list = list(col[0:num_cut].index)
        result = pd.DataFrame(list([features,test_auc_list])).T
        
        ## save dataframe
        
        df = df.append({
            'remain_features' : features,
            'cut_features' : cut_features_list,
            'test_AUC' : test_auc
        }, ignore_index=True)
        
        print_progressbar(num_round, i)
        print(f"  remain {len(result[0][i])} features")
    return df


## run RFE

In [11]:
result = RFE(new_train,y_train,new_test,y_test,num_round=10,num_cut=1)

Progress: [==                       ](10 %) (1/10)  remain 54 features
Progress: [=====                    ](20 %) (2/10)  remain 53 features
Progress: [=======                  ](30 %) (3/10)  remain 52 features
Progress: [==========               ](40 %) (4/10)  remain 51 features
Progress: [============             ](50 %) (5/10)  remain 50 features
Progress: [===============          ](60 %) (6/10)  remain 49 features
Progress: [=================        ](70 %) (7/10)  remain 48 features
Progress: [====================     ](80 %) (8/10)  remain 47 features
Progress: [======================   ](90 %) (9/10)  remain 46 features
Progress: [=========================](100 %) (10/10)
  remain 45 features


In [12]:
result

,remain_features,cut_features,test_AUC
0,"[[numeric1, numeric2, numeric3, numeric4, cate...",[category21],0.647619
1,"[[numeric1, numeric2, numeric3, numeric4, cate...",[category26],0.449524
2,"[[numeric1, numeric2, numeric3, numeric4, cate...",[category37],0.531429
3,"[[numeric1, numeric2, numeric3, numeric4, cate...",[category25],0.547619
4,"[[numeric1, numeric2, numeric3, numeric4, cate...",[category4],0.628571
5,"[[numeric1, numeric2, numeric3, numeric4, cate...",[category3],0.323810
6,"[[numeric1, numeric2, numeric3, numeric4, cate...",[category18],0.558095
7,"[[numeric1, numeric2, numeric3, numeric4, cate...",[category7],0.603810
8,"[[numeric1, numeric2, numeric3, numeric4, cate...",[category34],0.443810
9,"[[numeric1, numeric2, numeric3, numeric4, cate...",[category27],0.539048
